***
## Create a multi fasta file : 

In [10]:
import os
import pandas as pd
from Bio import SeqIO

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"

missing_dpos = pd.read_csv(f"{path_project}/full_missed_dpo.77_phages.tsv", sep = " ", header = 0)


In [11]:
with open(f"{path_project}/full_missing_dpos.multi.fasta", "w") as outfile :
    for _, row in missing_dpos.iterrows() : 
        path_fasta = f"{path_project}/annotation_classic/{row['Phage']}/{row['Phage']}.multi.annotated.faa"
        fasta_file = SeqIO.parse(path_fasta, "fasta")
        header = f"{row['Phage']} {row['CDS']},"
        for record in fasta_file : 
            if record.description.count(header) > 0 : 
                outfile.write(f">{record.description}\n{record.seq}\n")
            

    

> Move it to the server :

In [ ]:
rsync -avzhe ssh \
/media/concha-eloko/Linux/77_strains_phage_project/full_missing_dpos.multi.fasta \
conchae@garnatxa.srv.cpd:/home/conchae/PhageDepo_pdb

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/PhageDepo_pdb/full_missing_dpos_esmfold \
/media/concha-eloko/Linux/77_strains_phage_project/
